# Exploratory Data Analysis

In [42]:
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
import polars as pl
import plotly.express as px

# Load environment variables and create database engine
load_dotenv()
engine = create_engine(os.environ["DATABASE_URL"])


## Order and Revenue Analysis

To get an overview of our data, we can count the total number of orders and of items ordered in the online store. We can look at the schema of the polars dataframe to check if it correctly loaded the data. 

We might also be interested in some summary statistics, like the time range of our records, and total revenue in Brazilian reais (consisting of item prices + freight values, i.e. shipping costs).

In [43]:
with engine.connect() as conn:
    n_orders = conn.execute(text("SELECT COUNT(*) FROM olist.orders")).scalar()
    print("Orders in DB:", n_orders)

    n_ordered_items = conn.execute(text("SELECT COUNT(*) FROM olist.order_items")).scalar()
    print("Ordered items in DB:", n_ordered_items)

    df = pl.read_database("SELECT * FROM olist.order_items_expanded", engine)
    display(df.head())

    first = df.select(pl.col("order_purchase_timestamp").min()).item()
    last = df.select(pl.col("order_purchase_timestamp").max()).item()
    duration = last - first
    print(f"Order purchase timestamps range from {first} to {last} ({duration.days} days)")

    price_sum = df.select(pl.col("price").sum()).item()
    freight_sum = df.select(pl.col("freight_value").sum()).item()
    print(f"Total price of all order items: R$ {price_sum:,.0f}")
    print(f"Total freight value of all order items: R$ {freight_sum:,.0f}")
    print(f"Total revenue (price + freight): R$ {price_sum + freight_sum:,.0f}")

Orders in DB: 99441
Ordered items in DB: 112650


order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_delivered_customer_date,customer_id
str,i64,str,str,datetime[μs],"decimal[38,2]","decimal[38,2]",str,datetime[μs],datetime[μs],str
"""001ab0a7578dd66cd4b0a71f5b6e1e…",2,"""0b0172eb0fd18479d29c3bc122c058…","""5656537e588803a555b8eb41f07a94…",2018-01-04 02:33:42,24.89,17.63,"""delivered""",2017-12-27 00:38:47,2018-01-17 20:43:29,"""8bb3bef4e75a95524235cdc11a7331…"
"""001ab0a7578dd66cd4b0a71f5b6e1e…",3,"""0b0172eb0fd18479d29c3bc122c058…","""5656537e588803a555b8eb41f07a94…",2018-01-04 02:33:42,24.89,17.63,"""delivered""",2017-12-27 00:38:47,2018-01-17 20:43:29,"""8bb3bef4e75a95524235cdc11a7331…"
"""001ac194d4a326a6fa99b581e9a3d9…",1,"""dbaee28f4ee64465838a229582d775…","""ffc470761de7d0232558ba5e786e57…",2018-07-10 02:51:10,54.00,8.54,"""delivered""",2018-07-04 11:39:11,2018-07-12 17:48:49,"""f4f17ada91ef010cbde3892ae54e62…"
"""001b76dd48a5b1eee3e87778daa40d…",1,"""dbb67791e405873b259e4656bf9712…","""9c0e69c7bf2619675bbadf47b43f65…",2018-04-02 17:50:17,81.99,13.01,"""delivered""",2018-03-26 17:42:53,2018-04-06 18:36:23,"""a62843b08086d1629415a203a7fd41…"
"""001c85b5f68d2be0cb0797afc9e8ce…",1,"""84f456958365164420cfc80fbe4c7f…","""4a3ca9315b744ce9f8e93743614938…",2017-11-29 22:38:47,99.00,13.71,"""delivered""",2017-11-24 19:19:18,2017-12-22 18:37:40,"""48ed31e735f1c420ed6ca3637b7c74…"


Order purchase timestamps range from 2016-09-04 21:15:19 to 2018-09-03 09:06:57 (728 days)
Total price of all order items: R$ 13,591,644
Total freight value of all order items: R$ 2,251,910
Total revenue (price + freight): R$ 15,843,553


### Monthly revenue

Of interest to any online retailer is the monthly revenue. Here, we visualize it as a heatmap and the number of orders as a grouped line chart. Additionally, the plot of the cumulative revenue tells us the performance of sales over the years.  

In [45]:
with engine.connect() as conn:
    df_monthly_revenue = pl.read_database("SELECT * FROM olist.monthly_revenue", conn)
    df_monthly_revenue = df_monthly_revenue.with_columns(pl.col("datetime_month").dt.month().alias("month"))
    df_monthly_revenue = df_monthly_revenue.with_columns(pl.col("datetime_month").dt.year().alias("year"))
    df_monthly_revenue = df_monthly_revenue.with_columns(pl.col("total_revenue").cum_sum().alias("cumulative_revenue"))

    # As a heatmap
    df_pivot = df_monthly_revenue.to_pandas().pivot(values="total_revenue", index="year", columns="month")
    px.imshow(df_pivot.to_numpy(), 
              labels={"x": "Month", "y": "Year", "color": "Total Revenue"}, 
              x=df_pivot.columns.astype(str), 
              y=df_pivot.index.astype(str),
              title="Monthly Revenue Heatmap").show()

    px.line(df_monthly_revenue, x="datetime_month", y="cumulative_revenue", title="Cumulative Revenue Over Time", markers=True).show()

    px.line(df_monthly_revenue, x="month", y="n_orders", color="year", title="Monthly Number of Orders Over Time", markers=True).show()
